In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8n-cls.yaml')
model = YOLO('yolov8n-cls.pt')
model = YOLO('yolov8n-cls.yaml').load('yolov8n-cls.pt')

YOLOv8n-cls summary: 99 layers, 2719288 parameters, 2719288 gradients, 4.4 GFLOPs
YOLOv8n-cls summary: 99 layers, 2719288 parameters, 2719288 gradients, 4.4 GFLOPs
Transferred 158/158 items from pretrained weights


In [ ]:
results = model.train(data = r'C:\Capstone\data\classify',
                      model = r'C:\Capstone\runs\classify\cup_cls_3classes\weights\best.pt',
                      epochs = 100,
                      imgsz = (640, 640),
                      name = "cup_cls_3classes_resume",
                      resume = True,
                      patience = 10)

### Predict

In [4]:
predict_model = YOLO('yolov8n-cls.pt')
predict_model = YOLO(r'C:\Capstone\runs\classify\cup_cls_3classes_resume\weights\best.pt')
predict_model.names

{0: 'abnormal', 1: 'normal', 2: 'upsidedown'}

In [8]:
# Character Detect 모델 output이 Classification의 input이 되어야 함.

source = r'C:\Capstone\data\classify\test\normal'
predicted_results = predict_model(source, name='컵분류테스트', save=True, save_txt=False, show_labels=True, conf=0.9, show_conf=True, stream=True)
for r in predicted_results:
     boxes = r.boxes
     masks = r.masks
     probs = r.probs


image 1/25 C:\Capstone\data\classify\test\normal\BrfogTpk.jpg: 640x640 normal 1.00, abnormal 0.00, upsidedown 0.00, 22.0ms
image 2/25 C:\Capstone\data\classify\test\normal\CuHpXzfL2.jpg: 640x640 normal 1.00, upsidedown 0.00, abnormal 0.00, 20.0ms
image 3/25 C:\Capstone\data\classify\test\normal\EjdWtugj.jpg: 640x640 normal 1.00, upsidedown 0.00, abnormal 0.00, 20.0ms
image 4/25 C:\Capstone\data\classify\test\normal\IMG_0280.jpg: 640x640 normal 1.00, upsidedown 0.00, abnormal 0.00, 20.0ms
image 5/25 C:\Capstone\data\classify\test\normal\KeHdCcTU.jpg: 640x640 normal 1.00, abnormal 0.00, upsidedown 0.00, 19.0ms
image 6/25 C:\Capstone\data\classify\test\normal\QkmjxVNL.jpg: 640x640 normal 1.00, upsidedown 0.00, abnormal 0.00, 19.0ms
image 7/25 C:\Capstone\data\classify\test\normal\WIN_20240610_16_48_29_Pro5.jpg: 640x640 normal 1.00, abnormal 0.00, upsidedown 0.00, 20.0ms
image 8/25 C:\Capstone\data\classify\test\normal\WIN_20240610_16_49_11_Pro7.jpg: 640x640 normal 1.00, abnormal 0.00, up

### Semi Labelling

폴더1에서 캐릭터 디텍션 결과 별 txt 파일을 읽습니다.

txt 파일의 첫 번째 줄에서 한글 이름을 추출합니다.

동일한 이름의 이미지 파일을 캐릭터 디텍션 결과가 들어있는 폴더2에서 찾아 폴더3의 새로운 서브디렉토리에 저장합니다.

In [ ]:
import os
import shutil

def extract_top_label_from_txt(txt_file):
    """txt 파일의 첫 줄에서 라벨을 추출합니다."""
    with open(txt_file, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        label_name = first_line.split()[-1]
    return label_name

def copy_image_with_same_name_as_txt(txt_file, src_image_folder, dest_folder):
    """txt 파일의 이름과 동일한 이미지를 복사하되, 파일명에 라벨을 추가하여 저장합니다."""
    base_name_without_ext = os.path.splitext(os.path.basename(txt_file))[0]
    image_file_path = os.path.join(src_image_folder, base_name_without_ext + '.jpg')  # 이미지 확장자가 jpg라고 가정

    if os.path.exists(image_file_path):
        label = extract_top_label_from_txt(txt_file)
        label_folder = os.path.join(dest_folder, label)
        os.makedirs(label_folder, exist_ok=True)  # 폴더가 없다면 생성
        
        # 파일명에 라벨 추가
        dest_image_path = os.path.join(label_folder, base_name_without_ext + '-' + label + '.jpg')
        shutil.copy(image_file_path, dest_image_path)

def main():
    # 라벨 txt 디렉토리
    folder1 = r"F:\Project\3-2\webtoon_crawler_naver\Character_Classification\runs\classify\호랑이형님\labels"
    # Detect 결과 사진 디렉토리
    folder2 = r"F:\Project\3-2\webtoon_crawler_naver\webtoon_Classification\Character_Detection\runs\detect\호랑이형님_캐릭터_디텍션\crops\character"
    # 최종 결과 저장 디렉토리
    folder3 = r"F:\Project\3-2\data\호랑이형님\classified_character_folder"

    for txt_file in os.listdir(folder1):
        if txt_file.endswith('.txt'):
            txt_file_path = os.path.join(folder1, txt_file)
            copy_image_with_same_name_as_txt(txt_file_path, folder2, folder3)

if __name__ == "__main__":
    main()